In [134]:
import sys
sys.path.insert(0, "/home/aisummer/ssemenov_workspace/nlp-service")

import pandas as pd 
import json
import os
import time
import tqdm

from src.DocumentsParser.utils import DBS_DIR_DENSE_VECTORDB_NAME, DBS_DIR_SPARSE_VECTORDB_NAME
from src.evaluation_metrics import ReaderMetrics
from src.DocumentsSummarizer.Summarizer import SummarizerModule
from src.LLM_build.utils import llmConfig
from src.utils import DialogueState
from src.logger import Logger

### 1) Создаём объекты

In [2]:
reader = SummarizerModule()
config = DialogueState()
configLLM = llmConfig()

options = {
"num_keep": configLLM.num_keep,
"seed": configLLM.seed,
"num_predict": configLLM.num_predict,
"top_k": configLLM.top_k,
"top_p": configLLM.top_p,
"tfs_z": configLLM.tfs_z,
"typical_p": configLLM.typical_p,
"repeat_last_n": configLLM.repeat_last_n,
"temperature": configLLM.temperature,
"repeat_penalty": configLLM.repeat_penalty,
"presence_penalty": configLLM.presence_penalty,
"frequency_penalty": configLLM.frequency_penalty,
"mirostat": configLLM.mirostat,
"mirostat_tau": configLLM.mirostat_tau,
"mirostat_eta": configLLM.mirostat_eta,
"penalize_newline": configLLM.penalize_newline,
#"stop": config.stop,
"numa": configLLM.numa,
"num_ctx": configLLM.num_ctx,
"num_batch": configLLM.num_batch,
"num_gpu": configLLM.num_gpu,
"main_gpu": configLLM.main_gpu,
"low_vram": configLLM.low_vram,
"f16_kv": configLLM.f16_kv,
"vocab_only": configLLM.vocab_only,
"use_mmap": configLLM.use_mmap,
"use_mlock": configLLM.use_mlock,
"num_thread": configLLM.num_thread
}

### 2) Указываем инфо для данных

In [3]:
BENCHMARKS_INFO = {'sberquad': 'v1', 'squadv2': 'v1'}
banchmark_paths = {}
for name, version in BENCHMARKS_INFO.items():
    banchmark_paths[name] = {
        'table': f"../data/{name}/tables/{version}/benchmark.csv",
        'dense_db': f"../data/{name}/dbs/{version}/{DBS_DIR_DENSE_VECTORDB_NAME}",
        'sparse_db':  f"../data/{name}/dbs/{version}/{DBS_DIR_SPARSE_VECTORDB_NAME}"
    }

### 3) Загрузка датасета

In [4]:
# загрузить benchmark-датасет
benchmarks_df = {}
for name, bench_path in banchmark_paths.items():
    benchmarks_df[name] = pd.read_csv(banchmark_paths[name]['table'], sep=';')

# инициализировать класс с метриками
base_dir = '/home/aisummer/ssemenov_workspace/nlp-service'
metrics = ReaderMetrics(base_dir)

# logging
logger = Logger(False)
log = logger.get_logger(__name__)

Loading Meteor...
Loading ExactMatch


In [26]:
benchmarks_df['sberquad']

,question,answer,chunk_ids,contexts
0,чем представлены органические остатки?,известковыми выделениями сине-зелёных водорослей,[4974587056],['В протерозойских отложениях органические ост...
1,что найдено в кремнистых сланцах железорудной ...,"нитевидные водоросли, грибные нити",[4974587056],['В протерозойских отложениях органические ост...
2,что встречается в протерозойских отложениях?,органические остатки,[4974587056],['В протерозойских отложениях органические ост...
3,что относится к числу древнейших растительных ...,скопления графито-углистого вещества,[4974587056],['В протерозойских отложениях органические ост...
4,как образовалось графито-углистое вещество?,в результате разложения Corycium enigmaticum,[4974587056],['В протерозойских отложениях органические ост...
...,...,...,...,...
74251,Где используется классическая трёхуровневая си...,в рубиновом лазере.,[1898313107],['Классическая трёхуровневая система накачки р...
74252,какой инвестиционный банк входил в состав Amer...,Lehman Brothers,[4929086505],['Первая платёжная карта American Express появ...
74253,С каким альбомом был созвучен альбом Дэвида Бо...,Low,[1303094225],['Следующий альбом Heroes был во многом созвуч...
74254,В каком техасском оркестре выступал гитарист Л...,Light Crust Doughboys,[934655466],"['Одним из тех, на кого игра Данна произвела н..."


In [27]:
benchmarks_df['squadv2']

,question,answer,chunk_ids
0,When did Beyonce start becoming popular?,in the late 1990s,[621329309]
1,What areas did Beyonce compete in when she was...,singing and dancing,[621329309]
2,When did Beyonce leave Destiny's Child and bec...,2003,[621329309]
3,In what city and state did Beyonce grow up?,"Houston, Texas",[621329309]
4,In which decade did Beyonce become famous?,late 1990s,[621329309]
...,...,...,...
141964,What is the seldom used force unit equal to on...,sthène,[2990931136]
141965,What does not have a metric counterpart?,NaN,[2990931136]
141966,What is the force exerted by standard gravity ...,NaN,[2990931136]
141967,What force leads to a commonly used unit of mass?,NaN,[2990931136]


### 4) Предскажем ответы по вопросам из датасетов 

In [136]:
def get_relevant_ans(df: dict, reader: SummarizerModule, cnt: int, contexts: bool) -> list:
    relevant_ans = []
    questions = []
    context = []
    question = df['question']
    
    for i in tqdm.tqdm(range(cnt)):

        questions.append(question[i])

        if contexts:
            context.append(df['contexts'][i].strip('[]').strip("'"))
            contexts = df['contexts'][i].strip('[]').strip("'")
        else:
            contexts = ''

        reader.create_answer(config, configLLM, question[i], contexts.strip('[]').strip("'"))
        relevant_ans.append(config.answer)
    return questions, relevant_ans, context

### sberquad 

In [137]:
start_time_sberquad = time.time()

cnt = 5
questions_sberquad, predicted_sberquad, context_sberquad = get_relevant_ans(benchmarks_df['sberquad'], reader, cnt, True)
targets_sberquad = benchmarks_df['sberquad']['answer'].to_list()[:cnt]

time_execute_sberquad = time.time() - start_time_sberquad

q_and_a_sberquad = {
    'question': questions_sberquad,
    'predicted_answer': predicted_sberquad,
    'target': targets_sberquad,
    'context': context_sberquad
    }

 20%|██        | 1/5 [00:15<01:02, 15.70s/it]

 40%|████      | 2/5 [00:21<00:30, 10.14s/it]

 60%|██████    | 3/5 [00:27<00:15,  7.85s/it]

 80%|████████  | 4/5 [00:33<00:07,  7.26s/it]

100%|██████████| 5/5 [00:41<00:00,  8.33s/it]

In [138]:
df_sberquad = pd.DataFrame(q_and_a_sberquad)
df_sberquad.to_csv(f'sberquad_qa_{cnt}')
df_sberquad.head(10)

,question,predicted_answer,target,context
0,чем представлены органические остатки?,Органические остатки в протерозойских отложени...,известковыми выделениями сине-зелёных водорослей,В протерозойских отложениях органические остат...
1,что найдено в кремнистых сланцах железорудной ...,В cremnitsykh sglants (кремистых сланка) желез...,"нитевидные водоросли, грибные нити",В протерозойских отложениях органические остат...
2,что встречается в протерозойских отложениях?,"В Протерозое обнаружены органические остатки, ...",органические остатки,В протерозойских отложениях органические остат...
3,что относится к числу древнейших растительных ...,В протерозойских отложениях органические остат...,скопления графито-углистого вещества,В протерозойских отложениях органические остат...
4,как образовалось графито-углистое вещество?,Графито-углистым веществом называют остатки др...,в результате разложения Corycium enigmaticum,В протерозойских отложениях органические остат...


### squadv2

In [139]:
start_time_squadv2 = time.time()

questions_squadv2, predicted_squadv2, _ = get_relevant_ans(benchmarks_df['squadv2'], reader, cnt, False)
targets_squadv2 = benchmarks_df['squadv2']['answer'].to_list()[:cnt]

time_execute_squadv2 = time.time() - start_time_squadv2

q_and_a_squadv2 = {
    'question': questions_squadv2,
    'predicted_answer': predicted_squadv2,
    'target': targets_squadv2
}

 20%|██        | 1/5 [00:06<00:25,  6.35s/it]

 40%|████      | 2/5 [00:09<00:13,  4.60s/it]

 60%|██████    | 3/5 [00:15<00:10,  5.12s/it]

 80%|████████  | 4/5 [00:18<00:04,  4.47s/it]

100%|██████████| 5/5 [00:23<00:00,  4.66s/it]

In [140]:
df_squadv2 = pd.DataFrame(q_and_a_squadv2)
df_squadv2.to_csv(f'squadv2_qa_{cnt}')
df_squadv2.head(10)

,question,predicted_answer,target
0,When did Beyonce start becoming popular?,Beyoncé's popularity started to grow in the ea...,in the late 1990s
1,What areas did Beyonce compete in when she was...,"Когда была маленькой, Бейонсе соревновалась в ...",singing and dancing
2,When did Beyonce leave Destiny's Child and bec...,Beyoncé left Destiny's Child in 2004. The grou...,2003
3,In what city and state did Beyonce grow up?,"Бейонсе выросла в Хьюстоне, штат Техас. (Beyon...","Houston, Texas"
4,In which decade did Beyonce become famous?,"Beyoncé became famous in the early 2000s, spec...",late 1990s


### 6) Посчитаем метрики

In [141]:

def metric(predicted: list[str], targets: list[str], name:str) -> None:
    
    length = len(predicted)

    print('Dataset', name, '\n')
    rouge_score = metrics.rouge(predicted, targets[:length])
    print("rouge: ", rouge_score)

    bleu_score = metrics.bleu(predicted, targets[:length])
    print("bleu: ", bleu_score)

    meteor = metrics.meteor(predicted, targets[:length])
    print("meteor: ", meteor)

    exact_match = metrics.exact_match(predicted, targets[:length])
    print("exact_match: ", exact_match, '\n')

    # посчитать метрики 
    score = {
        'info': BENCHMARKS_INFO, 
        'params': options,
        'name': name,
        'rouge': rouge_score,
        'bleu': bleu_score,
        'meteor': meteor,
        'exact_match': exact_match
    }

    return score



score = metric(predicted_sberquad, targets_sberquad,'sberquad')

with open(f'../benchmark/logs/log_sberquad_{cnt}.txt', 'w') as f:
        f.write(f"info: {score['name']}\n")
        f.write(f"params: {score['params']}\n")
        f.write(f"Time of generation {cnt} answers: {time_execute_sberquad} seconds\n")
        f.write(f"Time of generation 1 answer: {time_execute_sberquad / cnt} seconds\n")
        f.write(f"rouge: {score['rouge']}\n")
        f.write(f"bleu: {score['bleu']}\n")
        f.write(f"meteor: {score['meteor']}\n")
        f.write(f"exact_match: {score['exact_match']}\n")
        f.write("\n")  

score = metric(predicted_squadv2, targets_squadv2, 'squadv2')

with open(f'../benchmark/logs/log_squadv2_{cnt}.txt', 'w') as f:
        f.write(f"info: {score['name']}\n")
        f.write(f"params: {score['params']}\n")
        f.write(f"Time of generation {cnt} answers: {time_execute_squadv2} seconds\n")
        f.write(f"Time of generation 1 answer: {time_execute_squadv2 / cnt} seconds\n")
        f.write(f"rouge: {score['rouge']}\n")
        f.write(f"bleu: {score['bleu']}\n")
        f.write(f"meteor: {score['meteor']}\n")
        f.write(f"exact_match: {score['exact_match']}\n")
        f.write("\n") 

Dataset sberquad 

rouge:  0.0
bleu:  0.08359
meteor:  0.36824
exact_match:  0.0 

Dataset squadv2 

rouge:  0.11818
bleu:  0.00775
meteor:  0.16575
exact_match:  0.0 

